Code die checkt of wij wav-files correct wegschrijven naar tf-records.


[link to original code](https://github.com/tensorflow/models/tree/master/research/audioset)

In [1]:
## Import packages
import os
import csv
import random
import numpy as np
import pandas as pd
from scipy.io import wavfile

import tensorflow as tf
from tensorflow import gfile
from tensorflow import logging

import vggish_input
import vggish_postprocess
import vggish_params
import vggish_slim

/home/hugo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
flags = tf.app.flags
FLAGS = flags.FLAGS
tf.app.flags.DEFINE_string('f', '', 'kernel')

flags.DEFINE_string(
    'audio_embedding_feature_name', 'audio_embedding',
    'Path to the VGGish checkpoint file.')

## De volgende bestanden komen uit het eerdere mapje 'models'. Nu heb ik ze in yt8m gezet.
## Deze zijn nodig om de embedding layer te maken
flags.DEFINE_string( 
    'pca_params', 'models/vggish_pca_params.npz',
    'Path to the VGGish PCA parameters file.')

flags.DEFINE_string(
    'checkpoint', 'models/vggish_model.ckpt',
    'Path to the VGGish checkpoint file.')

# flags.DEFINE_string(
#     'yt_checkpoint', 'models/youtube_model.ckpt',
#     'Path to the VGGish checkpoint file.')

In [3]:
filename = 'test_embedding/_D.tfrecord'    
record_iterator = tf.python_io.tf_record_iterator(filename)

## Met break: check alleen het eerste geluidsfragment uit de tf-record
embedding = None
    
for string_record in record_iterator:

    ## Take one example
    example = tf.train.SequenceExample()
    example.ParseFromString(string_record)

    embedding = example.feature_lists.feature_list['audio_embedding']

    break;

The youtube link to the first example

https://www.youtube.com/watch?v=_DHMdtRRJzE

In [4]:
#example
embedding

feature {
  bytes_list {
    value: "\330w\026\201\223rD\325\000U\000\234\312u\253\223\303\210\303\036\230/\303\356\301nM:~\205\207}\377\265\204~\303S}\247Wo`6@\263\333\230\227\257g*\237~z\212*\377~\021\201X\207B\335\302\324\227\213\234^\202O\211c\203<iZ\247S\202;\371\272}c\022\267k\377\207\036m\210\206\232\324k\000em\266^\2256X\341\201\333dt>\235d\243\365\313\346Z-\305!\262\361\346\200\345"
  }
}
feature {
  bytes_list {
    value: "\335vA\200\220nc\274\002l\000\262\233\335\225\207\265\212\204:}V\276\340\261f*MQ\210\264Y\301\247\216\223\354\177n\251\230\203\213}_n\343\203\273\314gi\232V<\254j\343\207[je\240\020\377u\201\2076\275.TX}{\304a_\221A\231c-\377\266e\234\206\242X\354_5O{\371\301\377\2453T%\276\276\301\014\177o\"\260R3i\207i\263\377\331\343}\000\302*^\314\3433\276"
  }
}
feature {
  bytes_list {
    value: "\301p%`\276~B\261\000\200(\272\220q\2325\263G\307d~\223~\340\304oy\207\272\270c0\260f\337\221\314\300Y\233^i_2d\225\257\327\325\330\212\247s\211A\377m\357\3178U\320\r1\277\

In [49]:
#byte_array = b'\xd8w\x16\x81\x93rD\xd5\x00U\x00\x9c\xcau\xab\x93\xc3\x88\xc3\x1e\x98/\xc3\xee\xc1nM:~\x85\x87}\xff\xb5\x84~\xc3S}\xa7Wo`6@\xb3\xdb\x98\x97\xafg*\x9f~z\x8a*\xff~\x11\x81X\x87B\xdd\xc2\xd4\x97\x8b\x9c^\x82O\x89c\x83<iZ\xa7S\x82;\xf9\xba}c\x12\xb7k\xff\x87\x1em\x88\x86\x9a\xd4k\x00em\xb6^\x956X\xe1\x81\xdbdt>\x9dd\xa3\xf5\xcb\xe6Z-\xc5!\xb2\xf1\xe6\x80\xe5'
byte_string = embedding.feature[0].bytes_list.value[0]

bs = np.fromstring(byte_string, dtype=np.uint8)
print(bs)
len(bs)

[216 119  22 129 147 114  68 213   0  85   0 156 202 117 171 147 195 136
 195  30 152  47 195 238 193 110  77  58 126 133 135 125 255 181 132 126
 195  83 125 167  87 111  96  54  64 179 219 152 151 175 103  42 159 126
 122 138  42 255 126  17 129  88 135  66 221 194 212 151 139 156  94 130
  79 137  99 131  60 105  90 167  83 130  59 249 186 125  99  18 183 107
 255 135  30 109 136 134 154 212 107   0 101 109 182  94 149  54  88 225
 129 219 100 116  62 157 100 163 245 203 230  90  45 197  33 178 241 230
 128 229]


/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


128

In [6]:
## Haal nu het wav-bestandje op dat van Youtube is gehaald
wavfile_path = 'test_embedding/youtube_D.wav'

In [24]:
## Deze code is van inference.py gehaald.
## Op het einde wordt er een nieuwe sequence example gemaakt

## Function that takes examples from wav-file as input and returns a sequence example
def getSequenceExample(examples_batch, labels, video_id=[b'_DHMdtRRJzE']):
    with tf.Graph().as_default(), tf.Session() as sess:

        # Define the model: load the checkpoint and locate input and output tensors
        # Input: [batch_size, num_frames, num_bands] 
        # where [num_frames, num_bands] represents log-mel-scale spectrogram
        # Output: embeddings
        vggish_slim.define_vggish_slim(training=False)
        vggish_slim.load_vggish_slim_checkpoint(sess, FLAGS.checkpoint)

#         pca_params = np.load(FLAGS.pca_params)
#         pca_matrix = pca_params[vggish_params.PCA_EIGEN_VECTORS_NAME]
#         pca_means = pca_params[vggish_params.PCA_MEANS_NAME].reshape(-1, 1)

        features_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_INPUT_TENSOR_NAME)
        embedding_tensor = sess.graph.get_tensor_by_name(
            vggish_params.VGGISH_OUTPUT_TENSOR_NAME)
        #vggish_slim.load_youtube_model(sess, FLAGS.yt_checkpoint)

        # Run inference and postprocessing
        [embedding_batch] = sess.run([embedding_tensor],
                                     feed_dict={features_tensor: examples_batch})

        # HK : Dit is echt anders
        postprocessed_batch = pproc.postprocess(embedding_batch)
#         np.dot(
#                 pca_matrix, (embedding_batch.T - pca_means)
#             ).T
        #print(postprocessed_batch)

#         num_frames = np.minimum(postprocessed_batch.shape[0], vggish_params.MAX_FRAMES)
#         data = vggish_postprocess.resize(postprocessed_batch, 0, vggish_params.MAX_FRAMES)
#         data = np.expand_dims(data, 0)
#         num_frames = np.expand_dims(num_frames, 0)

#         input_tensor = sess.graph.get_collection("input_batch_raw")[0]
#         num_frames_tensor = sess.graph.get_collection("num_frames")[0]

#         label_feat = tf.train.Feature(int64_list=tf.train.Int64List(value=labels))
#         videoid_feat = tf.train.Feature(bytes_list=tf.train.BytesList(value=video_id))
 
        seq_example = tf.train.SequenceExample(
            #context = tf.train.Features(feature={"labels": label_feat, "video_id": videoid_feat}),
            feature_lists = tf.train.FeatureLists(
                feature_list={
                    FLAGS.audio_embedding_feature_name:
                        tf.train.FeatureList(
                            feature=[
                                tf.train.Feature(
                                    bytes_list=tf.train.BytesList(
                                        value=[embedding.tobytes()]))
                                for embedding in postprocessed_batch
                            ]
                        )
                }
            )
        )
    return(seq_example,postprocessed_batch,embedding_batch)

In [25]:
## Maak eerst de examples
examples_batch = vggish_input.wavfile_to_examples(wavfile_path)
print("Examples shape: " + str(examples_batch.shape))

# Prepare a postprocessor to munge the model embeddings.
pproc = vggish_postprocess.Postprocessor(FLAGS.pca_params)

## Maak nu de embedding (met functie hierboven), pas PCA toe en maak er bytes van
seq_example, ppbatch, ebatch = getSequenceExample(examples_batch, [161])

Examples shape: (10, 96, 64)
INFO:tensorflow:Restoring parameters from models/vggish_model.ckpt


In [36]:
#print(seq_example)
#print(ppbatch)
print(ebatch.shape)

(10, 128)


In [68]:
byte_string_2 = seq_example.feature_lists.feature_list['audio_embedding'].feature[0].bytes_list.value[0]

In [11]:
# Vergelijk met de eerste:
# print(embedding.feature[0].bytes_list.value)

In [69]:
bs = np.fromstring(byte_string_2, dtype=np.uint8)
print(bs)
len(bs)

[181  17 128 109 233  56  76 117  32 184  70  91 178 110 113  61 168  91
 188 120  56 186 111 124 134 147 186 144 174 172 158  50 186 148 154 150
 130  88  61  62 154 170  23  29 116 166 129 170  82 229 133 107  70 160
  83 125 156  89 132 165 175 119  80 120 162 195  45  33 140 116 163 209
 208 243  75 194  84 230 126 149 113 255 111 128 126  46 206 208  84 172
 100 169  99 183  80  94  76  72 255  64 255  42 210  74 111 102 119 228
  94 245 177  99  72 116 188  68  56 199   0 227 213 255  63   7 247  66
 156 255]


/home/hugo/anaconda3/lib/python3.6/site-packages/ipykernel/__main__.py:1: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  if __name__ == '__main__':


128